In [20]:
import numpy as np
from matplotlib import pyplot as plt
import math

img = np.arange(0,255,4).reshape((8, 8))
print(img)

[[  0   4   8  12  16  20  24  28]
 [ 32  36  40  44  48  52  56  60]
 [ 64  68  72  76  80  84  88  92]
 [ 96 100 104 108 112 116 120 124]
 [128 132 136 140 144 148 152 156]
 [160 164 168 172 176 180 184 188]
 [192 196 200 204 208 212 216 220]
 [224 228 232 236 240 244 248 252]]


In [21]:
rand_img = np.random.randint(0, 255, size = (9,9))
print(rand_img)

[[166 148 239 144 241 207 235 203  32]
 [  3 130  64  69 134 215 199 152 125]
 [172 110  77  35   9 161  20  37 159]
 [235  95  59 233 227 211  60  87  20]
 [ 11 102 107 119  25  54 212 221 178]
 [174 166 158 187 129 112 192 207 120]
 [ 96  42 177  16 184  97 184 241 242]
 [225  91  51 188 146 172 138 174 151]
 [ 33  45  60 189 239  89  42 170  59]]


In [22]:
# correlation_mtx(rand_img)

In [23]:
# covariance matrix irrelevant for now
# np.cov(img, rowvar = False, bias = True)

In [24]:
def correlation_mtx(img):
    return np.corrcoef(img, rowvar = False)

In [25]:
def correlation_val(img):
    # from paper
    res = 0
    mean_y = np.mean(img,axis=0) # row mean
    mean_x = np.mean(img,axis=1) # col mean
    var_y = np.var(img,axis=0)
    var_x = np.var(img,axis=1)
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            res += (img[i,j] * (mean_x[i]*mean_y[j]) / (var_x[i]*var_y[j]))
    return res

In [26]:
def homogeneity(img):
    res = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            res += img[i,j] / (1 + abs(j - i))
    return res

In [47]:
def entropy(img):
    res = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            res += (img[i,j]+0.01) * math.log2(img[i,j]+0.01) # If pixel value is 0 log is infinite
            #  add
    return res * -1

In [40]:
from skimage.measure import shannon_entropy
entropy = shannon_entropy(img)

In [42]:
print(entropy)

6.0


In [28]:
def variance(img):
    return np.var(img)

In [29]:
def contrast(img):
    res = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            n = math.pow(i-j,2)
            res += img[j,i] * n
    return res

In [30]:
# Pyradiomics glrlm
def longRunEmphasis(img):
    return 1

In [31]:
def runePercentage(img):
    return 1

In [32]:
def var_mtx(img):
    return np.var(img,axis=0), np.var(img,axis=1)

In [36]:
import pandas as pd

data = pd.DataFrame(columns=[
    'target',
    'correlation',
    'homogeneity',
    'entropy',
    'variance',
    'contrast',
    'longRunEmphasis',
    'runePercentage'
])

target = 'normal'
# corr = correlation_mtx(img)
corr = correlation_val(img)
homo = homogeneity(img)
ent = entropy(img)
var = variance(img)
cont = contrast(img)
lre = longRunEmphasis(img)
rp = runePercentage(img)

row = {
    'target': target,
    'correlation': corr,
    'homogeneity': homo,
    'entropy': ent,
    'variance': var,
    'contrast': cont,
    'longRunEmphasis': lre,
    'runePercentage': rp
    }
data.append(row,ignore_index=True)

,target,correlation,homogeneity,entropy,variance,contrast,longRunEmphasis,runePercentage
0,normal,381.0,3140.1,-58611.159118,5460.0,84672.0,1,1
